In [ ]:
import sys
import os
repo_dir = os.path.dirname(os.getcwd())
sys.path.append(repo_dir)

In [ ]:
from utils.reproducibility import seed_everything
from models.lo import bins_lo, fast_bins_lo
from models.cm import ContinuousMixture
from torch.utils.data import DataLoader
from utils.datasets import load_debd
import numpy as np
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpus = None if device == 'cpu' else 1
print(device)

In [ ]:
train, valid, test = load_debd('binarized_mnist')

train_loader = DataLoader(train, batch_size=128, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid, batch_size=128)
test_loader = DataLoader(test, batch_size=128)

## Load the model

In [ ]:
path = ''
model = ContinuousMixture.load_from_checkpoint(path).to(device)
model.n_chunks = 64
model.missing = False
model.eval();

## Latent Optimisation

In [ ]:
max_epochs = 50
n_bins_list = [2**7, 2**8, 2**9, 2**10]
print(n_bins_list)

In [ ]:
for n_bins in n_bins_list:
    print('Evaluating using ' + str(n_bins) + ' bins..')
    seed_everything(42)
    z, log_w = bins_lo(model, n_bins, train_loader, valid_loader, max_epochs=max_epochs, lr=1e-3, device=device)
    print(model.eval_loader(test_loader, z, log_w, device=device).mean().item())